In [12]:
#import modules
import time
import json
import requests
from datetime import datetime
import pandas as pd 
import numpy as np

In [35]:
#!pip install pyecharts
print(time.time())

1580522339.35085


In [14]:
#define spider function
def catch_cityinfo():
    #scrape from Tensent
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=wuwei_ww_area_counts&callback=&_=%d'%int(time.time()*1000)
    city_data = requests.get(url=url).json()['data']
    city_data = json.loads(city_data)
    #return dataframe
    city_data = pd.DataFrame(city_data)  
    return city_data


In [15]:
#show data
citydata = catch_cityinfo()
citydata.head()
area_data = citydata

In [16]:
#import visualization modules
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.globals import ThemeType

In [20]:
#map visualization
area_map = Map()
area_map .add("",[list(z) for z in zip(list(area_data["area"]), list(area_data["confirm"]))], "china",is_map_symbol_show=False)
area_map.set_global_opts(title_opts=opts.TitleOpts(title="Map-疫情地图"),visualmap_opts=opts.VisualMapOpts(max_=3000,is_piecewise=True,
                pieces=[
                 {"min": "0", "max": "10", "label": "0-10", "color": "#FFC0CA"},
                 {"min": "10", "max": "100", "label": "10-100", "color": "#F08080"},
                 {"min": "100", "max": "1000", "label": "100-1000", "color": "#FF0000"},
                 {"min": "1000", "max": "10000", "label": ">1000", "color": "#8b0000"}
             ]))
area_map.render_notebook()

In [29]:
#data scrape according to date
def catch_daily():
    print(int(time.time()*1000))
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=wuwei_ww_cn_day_counts&callback=&_=%d'%int(time.time()*1000)
    data = json.loads(requests.get(url=url).json()['data'])
    data.sort(key=lambda x:x['date'])
    return data


In [30]:
daily_data = pd.DataFrame(catch_daily())
daily_data.head()

1580522193658


,date,confirm,suspect,dead,heal
0,01.13,41,0,1,0
1,01.14,41,0,1,0
2,01.15,41,0,2,5
3,01.16,45,0,2,8
4,01.17,62,0,2,12


In [23]:
#line chart
line = Line()
line.add_xaxis(list(daily_data["date"]))
line.add_yaxis("heal",list(daily_data["heal"]))
line.add_yaxis("dead", list(daily_data["dead"]))
line.set_global_opts(title_opts=opts.TitleOpts(title="Line-治愈与死亡趋势"))
line.render_notebook()


In [27]:
#bar chart of suspect and confirm
def bar():
    c = (
        Bar({"theme": ThemeType.MACARONS})
        .add_xaxis(list(daily_data["date"]))
        .add_yaxis("confirm", list(daily_data["confirm"]))
        .add_yaxis("suspect", list(daily_data["suspect"]))
        .set_global_opts(
            title_opts={"text": "Bar-患病人数增长趋势"}
        )
    )
    return c
bar().render_notebook()
